In [1]:
import os

In [2]:
print(os.getcwd())

c:\Users\ASUS\Desktop\ML-OPs\Kidney-Disease-Classification\research


In [3]:
os.chdir("c:/Users/ASUS/Desktop/ML-OPs/Kidney-Disease-Classification")
%pwd

'c:\\Users\\ASUS\\Desktop\\ML-OPs\\Kidney-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import sys
import zipfile
import gdown
from cnnClassifier import logging
from cnnClassifier.utils.common import get_size
from cnnClassifier import MyException

In [8]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from url.
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logging.info(f"Download data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise MyException(e, sys) from e
    
    def extract_zip_file(self) -> None:
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise MyException(e, sys) from e

[ 2025-06-22 14:28:03,325 ] INFO - common - yaml file: config\config.yaml loaded successfully.
[ 2025-06-22 14:28:03,328 ] INFO - common - yaml file: params.yaml loaded successfully.
[ 2025-06-22 14:28:03,334 ] INFO - common - Created directory at: artifacts
[ 2025-06-22 14:28:03,341 ] INFO - common - Created directory at: artifacts/data_ingestion
[ 2025-06-22 14:28:03,344 ] INFO - 3566406573 - Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip
[ 2025-06-22 14:28:03,389 ] DEBUG - connectionpool - Starting new HTTPS connection (1): drive.google.com:443
[ 2025-06-22 14:28:04,696 ] DEBUG - connectionpool - https://drive.google.com:443 "GET /uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3 HTTP/1.1" 200 None
[ 2025-06-22 14:28:05,273 ] DEBUG - connectionpool - https://drive.google.com:443 "GET /uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=a53817db-7543-45eb

[ 2025-06-22 14:28:27,401 ] INFO - common - Created directory at: artifact
